# BERT Testing


In [1]:
import os
from datetime import datetime

import matplotlib.pyplot as plt

import numpy as np

# Imports from PyTorch.
from torch import nn, Tensor, device, no_grad, manual_seed
from torch import max as torch_max
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Imports from aihwkit.
from aihwkit.nn import AnalogConv2d, AnalogLinear, AnalogSequential
import torch 
from aihwkit.optim import AnalogSGD
from aihwkit.simulator.configs import MappingParameter
from aihwkit.simulator.rpu_base import cuda
from datetime import datetime
from argparse import ArgumentParser
from collections import OrderedDict, defaultdict
from numpy import log10, logspace, argsort
from transformers.integrations import TensorBoardCallback

from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    Trainer,
    TrainingArguments,
    DefaultDataCollator,
)
from aihwkit.simulator.configs import (
    InferenceRPUConfig,
    NoiseManagementType,
    BoundManagementType,
)
from aihwkit.simulator.parameters.io import IOParametersIRDropT
from aihwkit.simulator.configs.utils import (
    WeightModifierType,
    BoundManagementType,
    WeightClipType,
    NoiseManagementType,
    WeightRemapType,
)

from torch import save as torch_save, load as torch_load
from torch.utils.tensorboard import SummaryWriter

from evaluate import load
from datasets import load_dataset

from aihwkit.simulator.configs import (
    InferenceRPUConfig,
    WeightModifierType, WeightClipType,
    WeightNoiseType,
    BoundManagementType,
    NoiseManagementType,
    WeightClipParameter,
    WeightModifierParameter,
    MappingParameter,
)

from aihwkit.simulator.presets import PresetIOParameters
from aihwkit.inference import PCMLikeNoiseModel, GlobalDriftCompensation, ReRamCMONoiseModel
from aihwkit.nn.conversion import convert_to_analog
from aihwkit.optim import AnalogSGD
#import wandb


In [10]:

def create_rpu_config(modifier_noise, tile_size=512, dac_res=-1, adc_res=-1):
    """Create RPU Config emulated typical PCM Device"""

    rpu_config = InferenceRPUConfig(
        clip=WeightClipParameter(type=WeightClipType.FIXED_VALUE, fixed_value=1.0),
        modifier=WeightModifierParameter(
            rel_to_actual_wmax=True, type=WeightModifierType.ADD_NORMAL, std_dev=modifier_noise
        ),
        mapping=MappingParameter(
            digital_bias=True,
            learn_out_scaling=True,
            weight_scaling_omega=1.0,
            out_scaling_columnwise=True,
            weight_scaling_columnwise=True,
            max_input_size=tile_size,
            max_output_size=0,
        ),
        forward=PresetIOParameters(is_perfect=True),
        noise_model=ReRamCMONoiseModel(g_max=90, g_min=10,
                                                        acceptance_range=0.2,
                                                        resistor_compensator=1.23,
                                                        single_device=True, 
                                                        drift_scale=1.0,
                                                        read_noise_scale=0.0,
                                                        prog_noise_scale=1.0),
        drift_compensation=None,
    )
    return rpu_config

In [23]:
MODEL_NAME = "csarron/bert-base-uncased-squad-v1"
def create_model(rpu_config):
    """Return Question Answering model and whether or not it was loaded from a checkpoint"""

    model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)
    model = convert_to_analog(model, rpu_config)
    #model.load_state_dict(torch.load("/u/mvc/bert_hwa2tpth"))
    model.remap_analog_weights()
    #model.load_state_dict(torch.load("/u/mvc/saved_chkpt.pth", map_location='cuda'))

    #print(model)
    return model

TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)
MAX_LENGTH = 320
DOC_STRIDE = 128
def preprocess_validation(dataset):
    """Preprocess the validation set"""
    # Some of the questions have lots of whitespace on the left,
    # which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space).
    # So we remove that
    # left whitespace
    dataset["question"] = [q.lstrip() for q in dataset["question"]]

    # Tokenize our dataset with truncation and maybe padding,
    # but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long,
    # each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_dataset = TOKENIZER(
        dataset["question"],
        dataset["context"],
        truncation="only_second",
        max_length=MAX_LENGTH,
        stride=DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context,
    # we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_dataset.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_dataset["example_id"] = []

    for i in range(len(tokenized_dataset["input_ids"])):
        # Grab the sequence corresponding to that example
        # (to know what is the context and what is the question).
        sequence_ids = tokenized_dataset.sequence_ids(i)
        context_index = 1

        # One example can give several spans,
        # this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_dataset["example_id"].append(dataset["id"][sample_index])

        # Set to None the offset_mapping that are not
        # part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_dataset["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_dataset["offset_mapping"][i])
        ]

    return tokenized_dataset


In [18]:
def postprocess_predictions(
    examples, features, raw_predictions, n_best_size=20, max_answer_length=30
):
    """Postprocess raw predictions"""
    features.set_format(type=features.format["type"], columns=list(features.features.keys()))
    all_start_logits, all_end_logits = raw_predictions

    # Map examples ids to index
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}

    # Create dict of lists, mapping example indices with corresponding feature indices
    features_per_example = defaultdict(list)

    for i, feature in enumerate(features):
        # For each example, take example_id, map to corresponding index
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill
    predictions = OrderedDict()

    print(
        f"Post-processing {len(examples)} example predictions "
        f"split into {len(features)} features."
    )

    # Loop over all examples
    for example_index, example in enumerate(examples):
        # Find the feature indices corresponding to the current example
        feature_indices = features_per_example[example_index]

        # Store valid answers
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            # This is what will allow us to map some the positions in our
            # logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are
                    # out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue

                    # Don't consider answers with a length
                    # that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    # Map the start token to the index of the start of that token in the context
                    # Map the end token to the index of the end of that token in the context
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]

                    # Add the answer
                    # Score is the sum of logits for the start and end position of the answer
                    # Include the text which is taken directly from the context
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                        }
                    )

        # If we have valid answers, choose the best one
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction,
            # we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Choose the best answer as the prediction for the current example
        predictions[example["id"]] = best_answer["text"]

    return predictions

def preprocess_train(dataset):
    """Preprocess the training dataset"""
    # Some of the questions have lots of whitespace on the left,
    # which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space).
    # So we remove that
    # left whitespace
    dataset["question"] = [q.lstrip() for q in dataset["question"]]

    # Tokenize our dataset with truncation and padding,
    # but keep the overflows using a stride. This results
    # in one example possibly giving several features when a context is long,
    # each of those features having a
    # context that overlaps a bit the context of the previous feature, the stride being the number
    # of overlapping tokens in the overlap.
    tokenized_dataset = TOKENIZER(
        dataset["question"],
        dataset["context"],
        truncation="only_second",
        max_length=MAX_LENGTH,
        stride=DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context,
    # we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_dataset.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to
    # character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_dataset.pop("offset_mapping")

    # Store start and end character positions for answers in context
    tokenized_dataset["start_positions"] = []
    tokenized_dataset["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_dataset["input_ids"][i]
        cls_index = input_ids.index(TOKENIZER.cls_token_id)

        # Grab the sequence corresponding to that example
        # (to know what is the context and what is the question).
        sequence_ids = tokenized_dataset.sequence_ids(i)

        # One example can give several spans, this
        # is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = dataset["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_dataset["start_positions"].append(cls_index)
            tokenized_dataset["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span
            # (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_dataset["start_positions"].append(cls_index)
                tokenized_dataset["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and
                # token_end_index to the two ends of the answer.
                # Note: we could go after the last offset
                # if the answer is the last word (edge case).
                while (
                    token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_dataset["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_dataset["end_positions"].append(token_end_index + 1)

    return tokenized_dataset

def create_datasets():
    """Load the SQuAD dataset, the tokenized version, and the validation set"""
    squad = load_dataset("squad")

    # Preprocessing changes number of samples, so we need to remove some columns so
    # the data updates properly
    tokenized_data = squad.map(
        preprocess_train, batched=True, remove_columns=squad["train"].column_names
    )
    eval_data = squad["validation"].map(
        preprocess_validation, batched=True, remove_columns=squad["validation"].column_names
    )

    return squad, tokenized_data, eval_data


In [21]:
def do_inference(model, trainer, squad, eval_data, writer, max_inference_time=1e6, n_times=9):
    """Perform inference experiment at weight noise level specified at runtime.
    SQuAD exact match and f1 metrics are captured in Tensorboard
    """

    # Helper functions
    def predict():
        # Perform inference + evaluate metric here
        raw_predictions = trainer.predict(eval_data)
        predictions = postprocess_predictions(
            squad["validation"], eval_data, raw_predictions.predictions
        )

        # Format to list of dicts instead of a large dict
        formatted_preds = [{"id": k, "prediction_text": v} for k, v in predictions.items()]

        out_metric = metric.compute(predictions=formatted_preds, references=ground_truth)

        return out_metric["f1"], out_metric["exact_match"]

    def write_metrics(f1, exact_match, t_inference):
        # Add information to tensorboard
        writer.add_scalar("val/f1", f1, t_inference)
        writer.add_scalar("val/exact_match", exact_match, t_inference)

    print("INFERENCE")
    metric = load("squad")

    ground_truth = [{"id": ex["id"], "answers": ex["answers"]} for ex in squad["validation"]]
    #f1, exact_match = predict()
    model.eval()


    t_inference_list = [1, 60*10, 3600, 3600 * 24, 3600 * 24*7, 3600 * 24 *30, 3600 * 24 *365, 3600 * 24 *365*2, 3600 * 24 *365*5, 3600 * 24 * 365 * 10] 
    f1_time = []
    # Get the initial metrics
    #f1, exact_match = predict()
    #write_metrics(f1, exact_match, 0.0)

    for t_inference in t_inference_list:
        print(t_inference)
        model.drift_analog_weights(t_inference)
        f1, exact_match = predict()
        f1_time.append(f1)
        print(f"Exact match: {exact_match: .2f}\t" f"F1: {f1: .2f}\t" f"Drift: {t_inference: .2e}")
    return f1_time 
        #write_metrics(f1, exact_match, t_inference)


In [6]:
def make_trainer(model, optimizer, tokenized_data):
    """Create the Huggingface Trainer"""
    training_args = TrainingArguments(
        output_dir="./",
        save_strategy="no",
        per_device_train_batch_size=4,
        per_device_eval_batch_size=256,
        dataloader_num_workers=16,
        num_train_epochs=3,
        weight_decay=0.001,
        no_cuda=False,
    )

    collator = DefaultDataCollator()

    log_dir = "logs/fit/25_JUN" 
    writer = SummaryWriter(log_dir=log_dir)

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=collator,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["validation"],
        tokenizer=TOKENIZER,
        optimizers=(optimizer, None),
        callbacks=[TensorBoardCallback(writer)],
    )

    return trainer, writer

In [ ]:

squad, tokenized_data, eval_data = create_datasets()

In [24]:
rpu_config = create_rpu_config(modifier_noise=0.0)
model = create_model(rpu_config)
optimizer = AnalogSGD(model.parameters(), lr=1e-3)


Some weights of the model checkpoint at csarron/bert-base-uncased-squad-v1 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
trainer, writer = make_trainer(model, optimizer, tokenized_data)

/tmp/ipykernel_3687053/1214499550.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
You are adding a <class 'transformers.integrations.integration_utils.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback


In [27]:
f1_times = do_inference(model, trainer, squad, eval_data,writer)

INFERENCE
1
Post-processing 10570 example predictions split into 11033 features.
Exact match:  0.30	F1:  8.65	Drift:  1.00e+00
600


Post-processing 10570 example predictions split into 11033 features.
Exact match:  11.78	F1:  31.27	Drift:  6.00e+02
3600


Post-processing 10570 example predictions split into 11033 features.
Exact match:  20.71	F1:  39.88	Drift:  3.60e+03
86400


Post-processing 10570 example predictions split into 11033 features.
Exact match:  74.84	F1:  84.59	Drift:  8.64e+04
604800


Post-processing 10570 example predictions split into 11033 features.
Exact match:  79.92	F1:  87.66	Drift:  6.05e+05
2592000


Post-processing 10570 example predictions split into 11033 features.
Exact match:  78.20	F1:  86.26	Drift:  2.59e+06
31536000


Post-processing 10570 example predictions split into 11033 features.
Exact match:  68.39	F1:  79.79	Drift:  3.15e+07
63072000


Post-processing 10570 example predictions split into 11033 features.
Exact match:  32.29	F1:  50.31	Drift:  6.31e+07
157680000


Post-processing 10570 example predictions split into 11033 features.
Exact match:  19.29	F1:  36.66	Drift:  1.58e+08
315360000


Post-processing 10570 example predictions split into 11033 features.
Exact match:  4.76	F1:  11.94	Drift:  3.15e+08


In [ ]:
rpu_config = create_rpu_config(modifier_noise=0.1)
squad, tokenized_data, eval_data = create_datasets()
model = create_model(rpu_config)
optimizer = AnalogSGD(model.parameters(), lr=1e-3)
trainer, writer = make_trainer(model, optimizer, tokenized_data)
do_inference(model, trainer, squad, eval_data,writer)

In [ ]:

def gen_rpu_config(noise_model=None):
    input_prec = 6
    output_prec = 8
    my_rpu_config = InferenceRPUConfig()
    my_rpu_config.mapping.digital_bias = True # do the bias of the MVM digitally
    #my_rpu_config.mapping.max_input_size = 256
    #my_rpu_config.mapping.max_output_size = 256
    my_rpu_config.forward = IOParametersIRDropT()
    my_rpu_config.noise_model = noise_model
    my_rpu_config.drift_compensation = None    #my_rpu_config.noise_model = PCMLikeNoiseModel(g_max=25.0)
    #my_rpu_config.drift_compensation = GlobalDriftCompensation()
    my_rpu_config.forward.ir_drop_g_ratio = 1.0 / 0.35 / (noise_model.g_max*1e-6) # change to 25w-6 when using PCM

    #my_rpu_config.drift_compensation = None
    my_rpu_config.modifier.std_dev = 0.06
    my_rpu_config.modifier.type = WeightModifierType.ADD_NORMAL
    
    my_rpu_config.forward.inp_res = 1 / (2**input_prec - 2)
    my_rpu_config.forward.out_res = 1 / (2**output_prec - 2)
    my_rpu_config.forward.is_perfect = True
    #my_rpu_config.forward.out_noise = 0.0 # Output on the current addition (?)
    my_rpu_config.forward.ir_drop = 1.0 # TODO set to 1.0 when activating IR drop effects
    my_rpu_config.forward.ir_drop_rs = 0.35 # Default: 0.15
    my_rpu_config.pre_post.input_range.enable = True
    
    #my_rpu_config.pre_post.input_range.manage_output_clipping = True
    my_rpu_config.pre_post.input_range.decay = 0.001
    my_rpu_config.pre_post.input_range.input_min_percentage = 0.95
    my_rpu_config.pre_post.input_range.output_min_percentage = 0.95
    #my_rpu_config.forward.noise_management = NoiseManagementType.ABS_MAX # Rescale back the output with the scaling for normalizing the input
    my_rpu_config.forward.bound_management = BoundManagementType.ITERATIVE
    my_rpu_config.clip.type = WeightClipType.LAYER_GAUSSIAN
    my_rpu_config.clip.sigma = 2.5
    my_rpu_config.forward.out_bound = 10.0  # quite restric
    return my_rpu_config

In [ ]:

g_max = 90
g_min = 10
prog_overshoot =1.23
single_device = True
acceptance_range = 0.2
reram_noise =ReRamCMONoiseModel(g_max=g_max, g_min=g_min,
                                                    acceptance_range=acceptance_range,
                                                    resistor_compensator=prog_overshoot,
                                                    single_device=single_device)
rpu_config= gen_rpu_config(reram_noise)
a_model = convert_to_analog(model, rpu_config=rpu_config).to(DEVICE)

In [ ]:
a_model.load_state_dict(torch.load(SAVE_PATH, map_location='cuda'))

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
n_rep = 5
t_inferences = [1, 60*10, 3600, 3600 * 24, 3600 * 24*7, 3600 * 24 *30, 3600 * 24 *365, 3600 * 24 *365*2, 3600 * 24 *365*5, 3600 * 24 * 365 * 10]
drifted_test_accs = torch.zeros(size=(len(t_inferences),n_rep))
a_model.eval()
for i,t in enumerate(t_inferences):
    for j in range(n_rep):
        a_model.drift_analog_weights(t)
        print("Drifted at t: ", t)
        _,_,err, accuracy = test_evaluation(validation_data=validation_data, model=a_model, criterion=criterion)
        drifted_test_accs[i, j] = accuracy
        print(f"Accuracy of the analog model: {accuracy:.2f}%")

In [ ]:
# BERT on SQUAD

In [ ]:
torch.save(drifted_test_accs, "/u/mvc/aihwkit/drift_compensation_NNs/hwa_vgg8_1T1R_10-90_baseline.th")